# Favorite-Longshot Pipeline

This notebook demonstrates the complete Favorite-Longshot (F-L) strategy pipeline:

1. **Connect** to Polymarket API
2. **Fetch** active markets
3. **Scan** for high-probability opportunities (85%+ favorites)
4. **Size** positions using Kelly Criterion
5. **Execute** paper trades
6. **Track** performance and P&L

## Background: The Favorite-Longshot Bias

Research by Snowberg & Wolfers shows that prediction markets systematically misprice extreme probabilities:
- **Longshots** (low probability events) are overpriced
- **Favorites** (high probability events) are underpriced

This strategy exploits the bias by buying heavily favored outcomes.

In [ ]:
# Setup: Add project root to path
import sys
sys.path.insert(0, '..')

import asyncio
from datetime import datetime, timezone, timedelta
from IPython.display import display, HTML
import pandas as pd

# Project imports
from src.adapters import PolymarketAdapter, Market
from src.core.config import Credentials
from src.strategies import FavoriteLongshotStrategy
from src.sizing.kelly import KellyCriterion, KellyFraction
from src.execution import PaperTrader, RiskManager, RiskLimits, CircuitBreakerManager
from src.execution.position_tracker import PositionTracker

print("Imports loaded successfully!")

## Step 1: Connect to Polymarket

In [ ]:
# Load credentials and connect
credentials = Credentials.from_env()
adapter = PolymarketAdapter(credentials=credentials)

await adapter.connect()
print(f"Connected: {adapter._client is not None}")

## Step 2: Fetch Active Markets

In [ ]:
# Fetch markets (including those nested in events)
markets = await adapter.get_all_markets(active_only=True, limit=300)
print(f"Fetched {len(markets)} active markets")

# Show sample
sample_df = pd.DataFrame([
    {
        'Question': m.question[:60] + '...' if len(m.question) > 60 else m.question,
        'YES': f"{m.yes_price:.1%}",
        'NO': f"{m.no_price:.1%}",
        'Volume': f"${m.volume:,.0f}" if m.volume else 'N/A',
        'Liquidity': f"${m.liquidity:,.0f}" if m.liquidity else 'N/A',
    }
    for m in markets[:10]
])
display(sample_df)

## Step 3: Configure F-L Strategy

In [ ]:
# Strategy parameters
MIN_PROBABILITY = 0.85  # Only consider 85%+ favorites
MIN_LIQUIDITY = 1000.0  # $1k minimum liquidity

strategy = FavoriteLongshotStrategy(
    min_probability=MIN_PROBABILITY,
    min_liquidity=MIN_LIQUIDITY,
)

print(f"Strategy configured:")
print(f"  Min probability: {MIN_PROBABILITY:.0%}")
print(f"  Min liquidity: ${MIN_LIQUIDITY:,.0f}")

## Step 4: Scan for Opportunities

In [ ]:
# Find F-L opportunities
opportunities = []

for market in markets:
    opp = strategy.check_market(market)
    if opp:
        opportunities.append(opp)

print(f"Found {len(opportunities)} opportunities out of {len(markets)} markets")
print(f"Hit rate: {len(opportunities)/len(markets):.1%}")

In [ ]:
# Display opportunities
if opportunities:
    opp_df = pd.DataFrame([
        {
            'Market': opp.market.question[:50] + '...' if len(opp.market.question) > 50 else opp.market.question,
            'Side': opp.side,
            'Price': f"{opp.price:.1%}",
            'Edge': f"{opp.edge:.2%}" if opp.edge else 'N/A',
            'Liquidity': f"${opp.liquidity:,.0f}" if opp.liquidity else 'N/A',
            'Hours to Exp': f"{opp.time_to_resolution:.0f}h" if opp.time_to_resolution else 'N/A',
        }
        for opp in opportunities[:15]
    ])
    display(opp_df)
else:
    print("No opportunities found with current parameters.")
    print("Try lowering MIN_PROBABILITY or MIN_LIQUIDITY.")

## Step 5: Kelly Criterion Sizing

In [ ]:
# Configure Kelly criterion
BANKROLL = 1000.0  # Starting capital
DEFAULT_EDGE = 0.02  # Assumed 2% edge on F-L opportunities

kelly = KellyCriterion(
    default_edge=DEFAULT_EDGE,
    fraction=KellyFraction.QUARTER,  # Conservative: 1/4 Kelly
)

print(f"Kelly configured:")
print(f"  Default edge: {DEFAULT_EDGE:.1%}")
print(f"  Kelly fraction: Quarter (25%)")
print(f"  Bankroll: ${BANKROLL:,.0f}")

In [ ]:
# Calculate bet sizes for top opportunities
if opportunities:
    sizing_data = []
    
    for opp in opportunities[:10]:
        bet = kelly.calculate(
            price=opp.price,
            bankroll=BANKROLL,
        )
        
        sizing_data.append({
            'Market': opp.market.question[:40] + '...',
            'Price': f"{opp.price:.1%}",
            'Kelly %': f"{bet.fraction:.2%}",
            'Bet Size': f"${bet.bet_size:.2f}",
            'Shares': f"{bet.bet_size / opp.price:.1f}",
            'Exp Growth': f"{bet.expected_growth:.4%}" if bet.expected_growth else 'N/A',
        })
    
    sizing_df = pd.DataFrame(sizing_data)
    display(sizing_df)

## Step 6: Paper Trading

In [ ]:
# Initialize paper trader
paper_trader = PaperTrader(initial_balance=BANKROLL)

print(f"Paper trader initialized with ${paper_trader.cash_balance:,.2f}")

In [ ]:
# Execute paper trades on top opportunities
trades_executed = 0
MIN_BET = 5.0  # Minimum $5 bet

for opp in opportunities[:5]:  # Top 5 opportunities
    bet = kelly.calculate(price=opp.price, bankroll=paper_trader.cash_balance)
    
    if bet.bet_size < MIN_BET:
        print(f"Skipping {opp.market.question[:30]}... (bet too small: ${bet.bet_size:.2f})")
        continue
    
    shares = bet.bet_size / opp.price
    
    try:
        trade = await paper_trader.execute_paper_trade(
            market_id=opp.market.id,
            side=opp.side,
            size=shares,
            price=opp.price,
            platform=opp.market.platform,
            question=opp.market.question[:50],
        )
        trades_executed += 1
        print(f"Traded: {opp.side} {shares:.1f} shares @ {opp.price:.1%} = ${bet.bet_size:.2f}")
    except ValueError as e:
        print(f"Trade failed: {e}")

print(f"\nExecuted {trades_executed} paper trades")

In [ ]:
# View open positions
positions = paper_trader.get_open_positions()

if positions:
    pos_df = pd.DataFrame([
        {
            'Market ID': pos.market_id[:20] + '...',
            'Side': pos.side,
            'Size': f"{pos.size:.1f}",
            'Entry': f"{pos.entry_price:.1%}",
            'Cost': f"${pos.cost_basis:.2f}",
        }
        for pos in positions
    ])
    display(pos_df)
else:
    print("No open positions")

In [ ]:
# Portfolio summary
summary = paper_trader.get_position_summary()

print("Portfolio Summary:")
print(f"  Cash Balance: ${summary['cash_balance']:,.2f}")
print(f"  Open Positions: {summary['open_positions']}")
print(f"  Total Cost Basis: ${summary['total_cost_basis']:,.2f}")
print(f"  Total Exposure: ${summary['total_cost_basis']:,.2f}")

## Step 7: Risk Management

In [ ]:
# Configure risk limits
position_tracker = PositionTracker()
risk_limits = RiskLimits(
    max_position_usd=100.0,      # Max $100 per position
    max_total_exposure=500.0,    # Max $500 total
    daily_loss_limit=100.0,      # Stop if down $100 in a day
    max_positions=20,            # Max 20 positions
)

risk_manager = RiskManager(risk_limits, position_tracker)

print("Risk limits configured:")
print(f"  Max position: ${risk_limits.max_position_usd}")
print(f"  Max exposure: ${risk_limits.max_total_exposure}")
print(f"  Daily loss limit: ${risk_limits.daily_loss_limit}")

In [ ]:
# Test risk checks
test_orders = [
    (50, 0.90, "Normal order"),
    (200, 0.90, "Exceeds position limit"),
    (30, 0.85, "Should pass"),
]

print("Risk Check Results:")
for size, price, desc in test_orders:
    check = risk_manager.check_new_order("test_market", size, price)
    status = "PASS" if check.passed else "BLOCK"
    print(f"  {desc}: {status}" + (f" ({check.reason})" if not check.passed else ""))

## Step 8: Circuit Breakers

In [ ]:
# Configure circuit breakers
circuit_breakers = CircuitBreakerManager()
circuit_breakers.add_loss_breaker(threshold=100, window_seconds=3600)  # $100 loss in 1 hour
circuit_breakers.add_consecutive_loss_breaker(threshold=5)  # 5 consecutive losses
circuit_breakers.add_error_breaker(threshold=10, window_seconds=300)  # 10 errors in 5 min

print("Circuit breakers configured:")
print(f"  Loss breaker: $100 in 1 hour")
print(f"  Consecutive loss breaker: 5 losses")
print(f"  Error breaker: 10 errors in 5 min")
print(f"\nTrading allowed: {circuit_breakers.allows_trading()}")

In [ ]:
# Simulate some losses to see breaker behavior
print("Simulating losses...")

circuit_breakers.record_loss(30)
print(f"After $30 loss: allows_trading={circuit_breakers.allows_trading()}")

circuit_breakers.record_loss(40)
print(f"After $40 more: allows_trading={circuit_breakers.allows_trading()}")

circuit_breakers.record_loss(35)
print(f"After $35 more (total $105): allows_trading={circuit_breakers.allows_trading()}")

status = circuit_breakers.get_status()
print(f"\nOpen breakers: {status['open_breakers']}")

## Step 9: Cleanup

In [ ]:
# Disconnect from API
await adapter.disconnect()
print("Disconnected from Polymarket API")

## Summary

This notebook demonstrated the complete F-L pipeline:

1. **Data**: Connected to Polymarket and fetched active markets
2. **Strategy**: Scanned for high-probability favorites (85%+)
3. **Sizing**: Used Kelly Criterion for optimal bet sizing
4. **Execution**: Paper traded top opportunities
5. **Risk**: Configured position limits and circuit breakers

### Key Parameters to Tune

| Parameter | Default | Description |
|-----------|---------|-------------|
| `MIN_PROBABILITY` | 85% | Minimum YES/NO price to consider |
| `MIN_LIQUIDITY` | $1,000 | Minimum market liquidity |
| `DEFAULT_EDGE` | 2% | Assumed edge on F-L opportunities |
| `KellyFraction` | QUARTER | How aggressive to bet (FULL, HALF, QUARTER, EIGHTH) |

### Next Steps

- Run with real data over time to track performance
- Tune parameters based on backtesting
- Add calibration analysis to refine edge estimates